# 수집한 데이터를 감성분석하여 결과를 DB에 추가 저장
Mongo DB에 저장되어 있는 네이버 뉴스 댓글 데이터를 꺼내와서 KoELECTRA 모델로 감성분석하고, 모델이 감성분석한 결과를 Mongo DB에 필드를 추가하여 새롭게 저장한다.


In [ ]:
!pip install transformers

In [1]:
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [2]:
# GPU 사용
device = torch.device("cuda")

In [ ]:
# 사전학습 모델 불러오기
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator").to(device)
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '.....'
# 파인튜닝된 모델 불러오기
model.load_state_dict(torch.load(f'{path}model(nsmc_aihub6).pt'))

<All keys matched successfully>

In [5]:
# 감성분석 결과를 리턴해주는 함수
def predict(text):
    encoded_review = tokenizer.encode_plus(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=128,
        padding=True,
        add_special_tokens=True
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output[0], dim=1)

    if str(prediction) == "tensor([1], device='cuda:0')":
        return '긍정'
    else:
        return '부정'

In [6]:
import requests
from pymongo import MongoClient
import json
from tqdm import tqdm
from pymongo.errors import BulkWriteError
from datetime import datetime, timedelta, timezone

client = MongoClient(
    host='....', 
    port=....,
    username='....',
    password='....',
    authSource='....')
db = client['....']

In [8]:
# Mongo DB에서 원하는 날짜 범위의 데이터만 추출하여 가져온다.
naver_list = list(db['....'].find({'date': {
    '$gte': datetime.strptime('2020-02-01T00:00:00+0900', '%Y-%m-%dT%H:%M:%S%z'),
    '$lt': datetime.strptime('2020-03-01T00:00:00+0900', '%Y-%m-%dT%H:%M:%S%z')
    }}, batch_size=1000))

In [9]:
len(naver_list)

46808

In [ ]:
naver_list[0]

In [ ]:
# 100개의 기사의 댓글데이터가 수집 될 때마다 db에 넣는다.
# 하나의 기사마다 일일이 업데이트 시키는 것보다 100개의 기사를 한번에 처리하는것이 더 빠르다.
# 기사 제목과 댓글을 감성분석하고 그 결과를 Mongo DB에 다시 저장하는 코드.
cnt = 0
arr = []
for i in tqdm(naver_list):
    cnt += 1
    # i['title'] 감성분석 코드
    i['sentiment_result'] = predict(i['title'])

    for comment in i['comments'] :
        # comment['text'] 감성분석 코드
        comment['sentiment_result'] = predict(comment['text'])

    arr.append(i)

    if cnt % 100 == 0 :
        print(cnt)
        try :
            db['.....'].insert_many(arr, ordered=False)
        except :
            pass
        arr = []
if arr :
    try :
        db['.....'].insert_many(arr,  ordered=False)
    except  :
        pass

In [ ]:
display(arr)


[{'_id': ObjectId('62b7de24d078569328d908e5'),
  'comments': [],
  'date': datetime.datetime(2022, 4, 1, 14, 58, 39),
  'offer': '뉴시스',
  'sentiment_result': '긍정',
  'text': '\n기사내용 요약"北 탄도미사일, 역내·세계 안보 환경에 명백한 위협"\n\n\n\n[서울=뉴시스] 북한 조선중앙TV가 북한이 지난 24일 김정은 북한 조선노동당 총비서 겸 국무위원장의 명령으로 신형 대륙간탄도미사일(ICBM) 화성 17형을 시험 발사했다고 25일 보도했다. (사진=조선중앙TV 캡처) 2022.03.25. photo@newsis.com *재판매 및 DB 금지[워싱턴=뉴시스]김난영 특파원 = 미국 재무부가 북한의 대량파괴무기(WMD)와 탄도미사일 프로그램에 연루된 5개 단체에 추가 제재를 가했다.재무부는 1일(현지시간) 해외자산통제실(OFAC)을 통해 유엔 안전보장이사회 결의안을 위반해 북한의 WMD 및 탄도미사일 프로그램 개발을 지원한 5개 단체를 제재하기로 했다고 밝혔다.재무부는 자료에서 "북한은 지난 2월26일과 3월4일에 두 번의 탄도미사일 시험 발사를 했다"라고 밝혔다. 이는 미국이 대륙간탄도미사일(ICBM) 실험으로 규정한 한국 시간 기준 2월27일·3월5일 시험 발사다.재무부는 해당 시험 발사가 북한이 개발 중인 신형 ICBM 시스템과 연관됐으며, 향후 ICBM 시험 수행에 대비한 역량 평가 차원이었다고 했다. 이와 함께 지난 3월24일 북한의 ICBM 시험 발사도 거론했다.이번 제재 대상에는 북한 군수산업부(MID) 하위조직인 로케트공업부를 비롯해 합장강무역회사, 조선승리산무역회사, 운천무역회사, 고려 등 수익 창출 관련 무역 기업 4곳이 포함됐다.재닛 옐런 재무장관은 보도자료에서 "북한의 도발적인 탄도미사일 실험은 역내, 그리고 세계의 안보 환경에 명백한 위협을 제기하고 유엔 안보리 결의안을 노골적으로 위반한다"라며 "북한